In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.legacy import data, datasets 
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# parameters
BATCH_SIZE = 64
learning_rate = 0.001
training_epochs = 10

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cuda


In [2]:
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)
trainset, valset, testset = datasets.SST.splits(TEXT, LABEL)

# data.Field.build_vocab() 라이브러리
# 문장 내 단어와 Integer index 를 매칭시키는 단어장(vocab)을 생성 == 워드 임베딩을 위한 Vocab 생성
# <UNK> = 0, <PAD> = 1 토큰도 추가.
# min_freq : 최소 5번 이상 등장한 단어들만 사전에 담겠다는 것. 
# 5번 미만으로 등장하는 단어는 UNK라는 토큰으로 대체

TEXT.build_vocab(trainset, min_freq=5)# TEXT 데이터를 기반으로 Vocab 생성
LABEL.build_vocab(trainset)# LABEL 데이터를 기반으로 Vocab 생성

# 매 배치마다 비슷한 길이에 맞춰 줄 수 있도록 iterator 정의
train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)

vocab_size = len(TEXT.vocab)
n_classes = 3

print("[TrainSet]: %d [ValSet]: %d [TestSet]: %d [Vocab]: %d [Classes] %d"
      % (len(trainset),len(valset), len(testset), vocab_size, n_classes))

[TrainSet]: 8544 [ValSet]: 1101 [TestSet]: 2210 [Vocab]: 3428 [Classes] 3


In [3]:
class GRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(GRU, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0)) # 첫번째 히든 스테이트를 0벡터로 초기화
        x, _ = self.gru(x, h_0)  # GRU의 리턴값은 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)
        h_t = x[:,-1,:] # (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
        self.dropout(h_t)
        logit = self.out(h_t)  # (배치 크기, 은닉 상태의 크기) -> (배치 크기, 출력층의 크기)
        return logit

    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [4]:
model = GRU(1, 256, vocab_size, 128, n_classes, 0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [5]:
# 모델 훈련
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(device), batch.label.to(device)
        y.data.sub_(1)  # 레이블 값을 0과 1로 변환
        optimizer.zero_grad()

        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

In [6]:
# 모델 평가 함수
def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(device), batch.label.to(device)
        y.data.sub_(1) # 레이블 값을 0과 1로 변환
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

In [7]:
BATCH_SIZE = 16
learning_rate = 0.001
best_of_best_acc = None # 최종 최고 accuracy model
for i in range(0, 4): # batch size  2의 배수로 128 까지 증가
    while(True):
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        train_iter, val_iter, test_iter = data.BucketIterator.splits(
                (trainset, valset, testset), batch_size=BATCH_SIZE,
                shuffle=True, repeat=False)

        # 모델 훈련
        best_val_accuracy = None
        counting = 0
        print('모델 훈련시작!')
        print(f'현재 모델 Batch Size: {BATCH_SIZE} Learning Rate: {learning_rate}')
        for e in range(1, training_epochs+1):
            train(model, optimizer, train_iter)
            val_loss, val_accuracy = evaluate(model, val_iter)
        
            print("[Epoch: %d] val loss : %5.2f | val accuracy : %5.2f" % (e, val_loss, val_accuracy))
        
            # 검증 오차가 가장 적은 최적의 모델을 저장
            if not best_val_accuracy or val_accuracy > best_val_accuracy:
                if not os.path.isdir("snapshot"):
                    os.makedirs("snapshot")
                torch.save(model.state_dict(), './snapshot/model1.pt')
                best_val_accuracy = val_accuracy
                counting = 0 # accuracy 가 증가하면 다시 counting 0으로 초기화
            else:
                counting += 1
                if counting == 2:
                    print('Accuracy 가 2 epoch 동안 증가하지 않았으므로 훈련은 멈춥니다.')
                    break
        print(f'현재 모델 훈련 끝 Accuracy: {best_val_accuracy}')
        print('----------------------------------------------------------------\n')

        # 검증 오차가 가장 적은 최적의 모델을 저장
        if not best_of_best_acc or best_val_accuracy > best_of_best_acc:
            torch.save(model.state_dict(), './snapshot/model2.pt') # 최종 모델은 model2에 저장
            best_of_best_acc = best_val_accuracy
            print('최종 모델의 Accuracy가 증가하여 model을 업데이트 했습니다.')
        
        # learning rate가 0.1 이면 빠져나가기
        if learning_rate >= 0.1:
            break
        learning_rate += 0.002
    BATCH_SIZE *= 2 # batch size 2배씩 증가
    learning_rate = 0.001 # learning rate 0.001 부터 시작하기 위해 초기화
print(f'최종 모델의 Accuracy: {best_of_best_acc}')


다음 모델 훈련시작!
현재 모델 Batch Size: 16 Learning Rate: 0.001
[Epoch: 1] val loss :  0.99 | val accuracy : 55.50
[Epoch: 2] val loss :  0.89 | val accuracy : 62.22
[Epoch: 3] val loss :  0.91 | val accuracy : 62.03
[Epoch: 4] val loss :  0.95 | val accuracy : 60.94
현재 모델 훈련 끝 Accuracy: {best_val_accuracy}
----------------------------------------------------------------

다음 모델 훈련시작!
현재 모델 Batch Size: 16 Learning Rate: 0.003
[Epoch: 1] val loss :  1.10 | val accuracy : 59.13
[Epoch: 2] val loss :  1.31 | val accuracy : 58.22
[Epoch: 3] val loss :  1.53 | val accuracy : 58.49
현재 모델 훈련 끝 Accuracy: {best_val_accuracy}
----------------------------------------------------------------

다음 모델 훈련시작!
현재 모델 Batch Size: 16 Learning Rate: 0.005
[Epoch: 1] val loss :  1.43 | val accuracy : 56.68
[Epoch: 2] val loss :  1.43 | val accuracy : 57.13
[Epoch: 3] val loss :  1.28 | val accuracy : 59.31
[Epoch: 4] val loss :  1.20 | val accuracy : 55.04
[Epoch: 5] val loss :  1.14 | val accuracy : 48.59
현재 모델 훈련 끝 A

In [8]:
model.load_state_dict(torch.load('./snapshot/model2.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print('테스트 오차: %5.2f | 테스트 정확도: %5.2f' % (test_loss, test_acc))

테스트 오차:  0.88 | 테스트 정확도: 64.30
